In [1]:
pip install kaggle

  Using cached bleach-6.3.0-py3-none-any.whl.metadata (31 kB)
  Using cached certifi-2025.10.5-py3-none-any.whl.metadata (2.5 kB)
  Using cached charset_normalizer-3.4.4-cp313-cp313-win_amd64.whl.metadata (38 kB)
  Using cached idna-3.11-py3-none-any.whl.metadata (8.4 kB)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached setuptools-80.9.0-py3-none-any.whl.metadata (6.6 kB)
  Using cached urllib3-2.5.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached webencodings-0.5.1-py2.py3-none-any.whl.metadata (2.1 kB)
Using cached certifi-2025.10.5-py3-none-any.whl (163 kB)
Using cached setuptools-80.9.0-py3-none-any.whl (1.2 MB)
Using cached urllib3-2.5.0-py3-none-any.whl (129 kB)
Using cached bleach-6.3.0-py3-none-any.whl (164 kB)
Using cached charset_normalizer-3.4.4-cp313-cp313-win_amd64.whl (107 kB)
Using cached idna-3.11-py3-none-any.whl (71 kB)
Using cached requests-2.32.5-py3-none-any.whl (64 kB)
Using cached webencodings-0.5.1-py2.py3-none-any.whl (11 kB)




[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import kaggle

In [3]:
# download dataset using kaggle api
!kaggle datasets download ankitbansal06/retail-orders -f orders.csv

Dataset URL: https://www.kaggle.com/datasets/ankitbansal06/retail-orders
License(s): CC0-1.0




  0%|          | 0.00/200k [00:00<?, ?B/s]
100%|██████████| 200k/200k [00:00<00:00, 199MB/s]


In [17]:
# if file is Zip then this is how we can unzip
import zipfile
zip_ref = zipfile.ZipFile('orders.csv.zip')
zip_ref.extractall() #exctract file to same location 
zip_ref.close() #close file

In [2]:
# read data from the file and handle null values
import pandas as pd
df = pd.read_csv('orders.csv',na_values=['Not Available','unknown'])
df.head(20)
df['Ship Mode'].unique()
df.columns = df.columns.str.strip().str.replace(' ','_').str.lower()
df.columns

Index(['order_id', 'order_date', 'ship_mode', 'segment', 'country', 'city',
       'state', 'postal_code', 'region', 'category', 'sub_category',
       'product_id', 'cost_price', 'list_price', 'quantity',
       'discount_percent'],
      dtype='object')

In [3]:
# derive new columns discount, sale price and profit
df['discount'] = df['list_price']*df['discount_percent']*.01
df.head(10)
df['sale_price'] = df['list_price']-df['discount']
df.head(5)
df['profit'] = df['sale_price']-df['cost_price']
df.head(5)

,order_id,order_date,ship_mode,segment,country,city,state,postal_code,region,category,sub_category,product_id,cost_price,list_price,quantity,discount_percent,discount,sale_price,profit
0,1,2023-03-01,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,FUR-BO-10001798,240,260,2,2,5.2,254.8,14.8
1,2,2023-08-15,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Chairs,FUR-CH-10000454,600,730,3,3,21.9,708.1,108.1
2,3,2023-01-10,Second Class,Corporate,United States,Los Angeles,California,90036,West,Office Supplies,Labels,OFF-LA-10000240,10,10,2,5,0.5,9.5,-0.5
3,4,2022-06-18,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Furniture,Tables,FUR-TA-10000577,780,960,5,2,19.2,940.8,160.8
4,5,2022-07-13,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Office Supplies,Storage,OFF-ST-10000760,20,20,2,5,1.0,19.0,-1.0


In [4]:
# convert order date from object data type to datetime
df['order_date'] = pd.to_datetime(df['order_date'])
df.dtypes
df['order_date']

0      2023-03-01
1      2023-08-15
2      2023-01-10
3      2022-06-18
4      2022-07-13
          ...    
9989   2023-02-18
9990   2023-03-17
9991   2022-08-07
9992   2022-11-19
9993   2022-07-17
Name: order_date, Length: 9994, dtype: datetime64[ns]

In [5]:
# drop cost price list price and discount percent columns
df.drop(columns= ['list_price','cost_price', 'discount_percent'], inplace=True)

In [6]:
from sqlalchemy import create_engine

In [8]:
# Load the data into sql server 
username = "postgres"
password = "admin1234"
host = "localhost"
port = "5432"
database = "orders_data"

engine = create_engine(f"postgresql+psycopg2://{username}:{password}@{host}:{port}/{database}")

# load dataframe into postgresSQL

table_name = "orders"
df.to_sql(table_name,engine,if_exists="append",index=False)
print(f"Data Successfully loaded into table'{table_name}' in database '{database}'.")

Data Successfully loaded into table'orders' in database 'orders_data'.
